<a href="https://colab.research.google.com/github/dainaklis/PySpark/blob/main/Darbdaviai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORT AND INSTAL**

In [ ]:
!pip install pyspark
!pip install --upgrade matplotlib

In [ ]:
from pyspark.sql import functions as F, Row, SparkSession
from pyspark.sql.functions import expr, col
from pyspark.sql.functions import *
from pyspark.sql import  SQLContext
from pyspark.sql import types 
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import pyspark.sql.functions as func

# **ANALIZUOJAM**

In [ ]:
spark = SparkSession.builder.appName("MySecondApp").getOrCreate()
st = spark.read.csv("AtvejaiPagal.csv", header="true", inferSchema=True)

need = st.select("apskritis","darbuotojai","atvejai_viso","atvejai_14d","norma_14d","testai_14d","veiklos_pobudis")
need.show(truncate=False, n=10)
need.printSchema()

+-------------------+-----------+------------+-----------+---------+----------+--------------------------------------------------+
|apskritis          |darbuotojai|atvejai_viso|atvejai_14d|norma_14d|testai_14d|veiklos_pobudis                                   |
+-------------------+-----------+------------+-----------+---------+----------+--------------------------------------------------+
|Utenos apskr.      |238        |83          |27         |3        |154       |Vielos gaminių, grandinių ir spyruoklių gamyba    |
|Vilniaus apskr.    |160        |54          |22         |3        |37        |Viešųjų ryšių ir komunikacijos veikla             |
|Alytaus apskr.     |159        |72          |19         |4        |83        |Buitinių elektrinių aparatų ir prietaisų gamyba   |
|Vilniaus apskr.    |1904       |840         |27         |24       |767       |Ligoninių veikla                                  |
|Marijampolės apskr.|279        |47          |16         |4        |45        |Liet

In [ ]:
need.describe().show(truncate=False)

+-------+--------------+-----------------+------------------+------------------+-----------------+-----------------+---------------------------------------------------------+
|summary|apskritis     |darbuotojai      |atvejai_viso      |atvejai_14d       |norma_14d        |testai_14d       |veiklos_pobudis                                          |
+-------+--------------+-----------------+------------------+------------------+-----------------+-----------------+---------------------------------------------------------+
|count  |3266          |3342             |3342              |3342              |3342             |3340             |3331                                                     |
|mean   |null          |160.6376421304608|28.727707959305803|2.3497905445840814|2.826750448833034|27.75958083832335|null                                                     |
|stddev |null          |439.9387716182878|80.10760911103677 |3.138244211264703 |5.315451279211243|81.14866790225219|null     

In [ ]:
need.select("apskritis", "darbuotojai","atvejai_viso").summary("count", "mean", "max").show(truncate=False)

In [ ]:
sumApskriciu = need.groupBy("apskritis").sum()

# Apskritys, kurios turi daugiausia darbuotoju
sumApskriciu.sort(desc("sum(darbuotojai)")).limit(10).show(truncate=False)

# Apskritys, kur buvo daugiausiai atveju
sumApskriciu.sort(desc("sum(atvejai_viso)")).limit(10).show(truncate=False)

# Apskritys, kur buvo daugiausiai atveju 14d
sumApskriciu.sort(desc("sum(atvejai_14d)")).limit(10).show(truncate=False)


+-------------------+----------------+-----------------+----------------+--------------+---------------+
|apskritis          |sum(darbuotojai)|sum(atvejai_viso)|sum(atvejai_14d)|sum(norma_14d)|sum(testai_14d)|
+-------------------+----------------+-----------------+----------------+--------------+---------------+
|Vilniaus apskr.    |266413          |44283            |2975            |4189          |39896          |
|Kauno apskr.       |97757           |17460            |1309            |1396          |15515          |
|Klaipėdos apskr.   |57073           |11332            |1347            |1687          |14719          |
|Šiaulių apskr.     |27579           |5225             |570             |611           |5657           |
|Panevėžio apskr.   |20262           |4164             |275             |218           |3827           |
|Telšių apskr.      |19995           |3751             |443             |497           |4653           |
|Marijampolės apskr.|13243           |2570             

In [ ]:
# Grupuojam pagal veiklos pabudi
sumVeiklos = need.groupBy("veiklos_pobudis").sum()

# VEIKLOS, kurios turi daugiausia darbuotoju
sumVeiklos.sort(desc("sum(darbuotojai)")).limit(10).show(truncate=False)

# VEIKLOS, kur buvo daugiausiai atveju
sumVeiklos.sort(desc("sum(atvejai_viso)")).limit(10).show(truncate=False)

# VEIKLOS, kur buvo daugiausiai atveju 14d
sumVeiklos.sort(desc("sum(atvejai_14d)")).limit(10).show(truncate=False)

+---------------------------------------------------------------------------------------------+----------------+-----------------+----------------+--------------+---------------+
|veiklos_pobudis                                                                              |sum(darbuotojai)|sum(atvejai_viso)|sum(atvejai_14d)|sum(norma_14d)|sum(testai_14d)|
+---------------------------------------------------------------------------------------------+----------------+-----------------+----------------+--------------+---------------+
|Ligoninių veikla                                                                             |36504           |10863            |301             |481           |9721           |
|Mažmeninė prekyba nespecializuotose parduotuvėse, kuriose vyrauja maistas, gėrimai ir tabakas|32361           |4602             |255             |417           |5283           |
|Vidurinis ugdymas                                                                            |16896     

In [ ]:
# AVERAGE
avgAllVak = need.groupBy("veiklos_pobudis").avg()
avgAllVak.sort(desc("avg(darbuotojai)")).limit(10).show(truncate=False)

+--------------------------------------------------------------------------+------------------+------------------+-----------------+------------------+------------------+
|veiklos_pobudis                                                           |avg(darbuotojai)  |avg(atvejai_viso) |avg(atvejai_14d) |avg(norma_14d)    |avg(testai_14d)   |
+--------------------------------------------------------------------------+------------------+------------------+-----------------+------------------+------------------+
|Pašto pagal įpareigojimą teikti universaliąsias paslaugas atliekama veikla|4029.0            |619.0             |20.0             |48.0              |214.0             |
|Keleivių vežimas miesto arba priemiestiniu sausumos transportu            |1974.0            |476.0             |15.0             |25.0              |637.0             |
|Kitų, niekur kitur nepriskirtų, cheminių medžiagų gamyba                  |1707.0            |240.0             |4.0              |22.0         